# 04 综合案例：智能商品搜索

本 Notebook 串联 Demo 1-3 的技术，构建一个端到端的智能商品搜索系统。

```
CSV (Demo1 + Demo3)
    --> Daft: Clean & Transform
    --> LanceDB: Store (structured, no vectors)
    --> Daft: read_lance + embed_text
    --> LanceDB: Store (with vectors)
    --> Search & Recommend
```

三个工具各司其职：
- **Daft**：数据处理引擎（清洗、转换、嵌入生成）
- **LanceDB**：数据湖 + 向量搜索引擎（存储、索引、查询）
- **Ray**（Demo 2）：分布式加速引擎（大规模场景下切换 Runner 即可）

## 1. 数据整合与清洗

In [ ]:
import daft
from daft import col, lit

### 读取产品数据（Demo 1）

In [ ]:
df_products = daft.read_parquet("../../demo1_daft/data/products.parquet")
print(f"产品数据: {df_products.count_rows()} 条")
df_products.limit(3).show()

### 读取评论数据（Demo 3）

In [ ]:
df_reviews = daft.read_csv("../data/reviews.csv")
print(f"评论数据: {df_reviews.count_rows()} 条")
df_reviews.limit(3).show()

### 产品数据清洗

Demo 1 的数据有 ~2% 重复、~5% description 缺失、~3% brand 缺失。

In [ ]:
total = df_products.count_rows()

df_products_clean = (
    df_products
    .distinct("product_id")
    .where(col("description").not_null())
    .with_column("brand", col("brand").fill_null("未知品牌"))
    .with_column("rating", col("rating").fill_null(0.0))
    .where(col("price") > 0)
)

print(f"清洗前: {total} 条")
print(f"清洗后: {df_products_clean.count_rows()} 条")

### 构建搜索文本

拼接品牌、子类别、描述为一个 `search_text` 列，让搜索时品牌和类别信息也参与语义匹配。

In [ ]:
df_products_final = df_products_clean.with_column(
    "search_text",
    col("brand") + lit(" ") + col("subcategory") + lit(" ") + col("description"),
)

df_products_final.select("product_id", "search_text").limit(3).show()

### 评论数据清洗

In [ ]:
df_reviews_clean = (
    df_reviews
    .where(col("review").not_null())
    .where(col("review").length() > 5)
)

print(f"清洗前: {df_reviews.count_rows()} 条")
print(f"清洗后: {df_reviews_clean.count_rows()} 条")

## 2. LanceDB 作为数据湖

LanceDB 不仅是向量搜索引擎，也是数据湖。先把清洗后的结构化数据（无向量）存入 LanceDB，作为统一存储层。后续嵌入生成后再写回带向量的版本。

In [ ]:
import lancedb

db = lancedb.connect("../lancedb_data")

# 写入产品数据（无向量）
products_pd = df_products_final.to_pandas()
products_raw = db.create_table("products_raw", products_pd.to_dict("list"), mode="overwrite")
print(f"产品表: {products_raw.count_rows()} 条")

# 写入评论数据（无向量）
reviews_pd = df_reviews_clean.to_pandas()
reviews_raw = db.create_table("reviews_raw", reviews_pd.to_dict("list"), mode="overwrite")
print(f"评论表: {reviews_raw.count_rows()} 条")

print(f"\n数据库表: {db.table_names()}")

### 用 Daft 直接读取 LanceDB 数据

Daft 可以通过 `read_lance()` 直接读取 LanceDB 底层的 Lance 文件做分析，无需经过 LanceDB API。

In [ ]:
df_from_lance = daft.read_lance("../lancedb_data/products_raw.lance")
print("从 LanceDB 读取产品数据做分析:")
df_from_lance.groupby("category").agg(
    col("product_id").count().alias("count"),
    col("price").mean().alias("avg_price"),
).sort("count", desc=True).show()

## 3. 嵌入生成与写回

从 LanceDB 读出数据，用 Daft `embed_text` 生成嵌入，再写回 LanceDB 带向量的版本。

**需要 SiliconFlow API Key。**

In [ ]:
import os
from daft.functions.ai import embed_text

assert os.environ.get("OPENAI_API_KEY"), "请设置环境变量 OPENAI_API_KEY"

daft.set_provider(
    "openai",
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1"),
)

EMBEDDING_MODEL = "Qwen/Qwen3-Embedding-4B"
EMBEDDING_DIM = 1024
print(f"Provider: {daft.current_provider()}")

### 为产品生成嵌入

In [ ]:
df_products_lance = daft.read_lance("../lancedb_data/products_raw.lance")

df_products_embedded = df_products_lance.with_column(
    "vector",
    embed_text(col("search_text"), model=EMBEDDING_MODEL, dimensions=EMBEDDING_DIM),
)

print("正在生成产品嵌入...")
products_with_vec = df_products_embedded.to_pandas()
print(f"完成: {len(products_with_vec)} 条，向量维度 {len(products_with_vec['vector'].iloc[0])}")

### 为评论生成嵌入

In [ ]:
df_reviews_lance = daft.read_lance("../lancedb_data/reviews_raw.lance")

df_reviews_embedded = df_reviews_lance.with_column(
    "vector",
    embed_text(col("review"), model=EMBEDDING_MODEL, dimensions=EMBEDDING_DIM),
)

print("正在生成评论嵌入...")
reviews_with_vec = df_reviews_embedded.to_pandas()
print(f"完成: {len(reviews_with_vec)} 条，向量维度 {len(reviews_with_vec['vector'].iloc[0])}")

### 写回 LanceDB（带向量版本）

In [ ]:
products_search = db.create_table("products", products_with_vec.to_dict("list"), mode="overwrite")
reviews_search = db.create_table("reviews", reviews_with_vec.to_dict("list"), mode="overwrite")

print(f"产品搜索表: {products_search.count_rows()} 条")
print(f"评论搜索表: {reviews_search.count_rows()} 条")
print(f"\n数据库表: {db.table_names()}")

## 4. 为什么需要 LanceDB？暴力搜索 vs LanceDB

向量搜索也可以用 numpy 手动实现。下面对比两种方式，展示 LanceDB 的价值。

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    base_url=os.environ.get("OPENAI_BASE_URL", "https://api.siliconflow.cn/v1"),
)


def get_query_vector(query: str) -> list[float]:
    """生成查询文本的嵌入向量"""
    response = client.embeddings.create(input=[query], model=EMBEDDING_MODEL)
    return response.data[0].embedding

### 方式一：numpy 暴力搜索

In [ ]:
import numpy as np
import time

query = "拍照好的手机"
query_vec = np.array(get_query_vector(query))

# 暴力搜索：加载所有向量，逐一计算距离
all_data = products_search.to_pandas()
all_vectors = np.array(all_data["vector"].tolist())

start = time.time()
distances = np.linalg.norm(all_vectors - query_vec, axis=1)
top_indices = np.argsort(distances)[:5]
brute_time = time.time() - start

print(f"暴力搜索: '{query}' ({brute_time*1000:.2f} ms)\n")
for idx in top_indices:
    row = all_data.iloc[idx]
    print(f"  [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")
    print(f"    {row['description'][:50]}")
    print(f"    距离: {distances[idx]:.4f}")

### 方式二：LanceDB 搜索（同样的查询，2 行代码）

In [ ]:
start = time.time()
results = products_search.search(query_vec.tolist()).limit(5).to_pandas()
lance_time = time.time() - start

print(f"LanceDB 搜索: '{query}' ({lance_time*1000:.2f} ms)\n")
for _, row in results.iterrows():
    print(f"  [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")
    print(f"    {row['description'][:50]}")
    print(f"    距离: {row['_distance']:.4f}")

### 混合搜索对比

LanceDB 的真正优势在混合搜索：向量相似度 + 标量过滤一步完成。暴力搜索需要手动过滤再计算距离。

In [ ]:
# 暴力混合搜索：手动过滤 + 手动计算距离 + 手动排序
mask = (all_data["category"] == "电子产品") & (all_data["price"] < 1000)
filtered_vectors = all_vectors[mask]
filtered_data = all_data[mask].reset_index(drop=True)
distances_filtered = np.linalg.norm(filtered_vectors - query_vec, axis=1)
top_filtered = np.argsort(distances_filtered)[:5]

print("暴力混合搜索（手动过滤 + 排序）:")
for idx in top_filtered:
    row = filtered_data.iloc[idx]
    print(f"  [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")

# LanceDB 混合搜索：一行 .where() 搞定
print("\nLanceDB 混合搜索:")
results = (
    products_search
    .search(query_vec.tolist())
    .where("category = '电子产品' AND price < 1000")
    .limit(5)
    .to_pandas()
)
for _, row in results.iterrows():
    print(f"  [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")

## 5. 智能搜索与推荐

基于 LanceDB 中的两张向量表，实现商品搜索、相似推荐和跨维度组合搜索。

### 商品语义搜索

In [ ]:
import pandas as pd


def search_products(query: str, top_k: int = 5) -> pd.DataFrame:
    """语义搜索商品"""
    query_vec = get_query_vector(query)
    return products_search.search(query_vec).limit(top_k).to_pandas()


queries = ["轻薄续航好的手机", "适合敏感肌的护肤品", "保暖的冬季外套"]
for q in queries:
    results = search_products(q, top_k=3)
    print(f"搜索: '{q}'")
    for _, row in results.iterrows():
        print(f"  [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}  {row['description'][:40]}")
    print()

### 相似商品推荐

给定一个商品，用它的向量找到最相似的商品——"看了这个商品的人还看了..."。

In [ ]:
def recommend_similar(product_id: str, top_k: int = 5) -> tuple[pd.Series, pd.DataFrame]:
    """根据商品 ID 推荐相似商品"""
    target = products_search.search().where(f"product_id = '{product_id}'").limit(1).to_pandas()
    if target.empty:
        raise ValueError(f"商品 {product_id} 不存在")
    target_row = target.iloc[0]
    results = (
        products_search
        .search(target_row["vector"])
        .where(f"product_id != '{product_id}'")
        .limit(top_k)
        .to_pandas()
    )
    return target_row, results


# 取一个电子产品做演示
sample = products_search.search().where("category = '电子产品'").limit(1).to_pandas()
demo_id = sample.iloc[0]["product_id"]

target, similar = recommend_similar(demo_id)
print(f"目标: [{target['subcategory']}] {target['brand']} - ¥{target['price']:.0f}")
print(f"  {target['description'][:60]}")
print(f"\n相似商品:")
for i, (_, row) in enumerate(similar.iterrows(), 1):
    print(f"  {i}. [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")
    print(f"     {row['description'][:50]}")

### 跨维度组合搜索

同时搜索产品和评论——产品描述告诉你"商家说了什么"，评论告诉你"用户怎么说"。

In [ ]:
def combined_search(query: str, top_k: int = 3) -> None:
    """组合搜索：同时搜索产品和评论"""
    query_vec = get_query_vector(query)
    products = products_search.search(query_vec).limit(top_k).to_pandas()
    reviews = reviews_search.search(query_vec).limit(top_k).to_pandas()

    label_map = {1: "正面", 0: "负面"}
    print(f"=== 搜索: '{query}' ===")
    print(f"\n--- 匹配商品 ---")
    for i, (_, row) in enumerate(products.iterrows(), 1):
        print(f"  {i}. [{row['subcategory']}] {row['brand']} - ¥{row['price']:.0f}")
        print(f"     {row['description'][:50]}")
    print(f"\n--- 相关评论 ---")
    for i, (_, row) in enumerate(reviews.iterrows(), 1):
        sentiment = label_map.get(row['label'], '未知')
        print(f"  {i}. [{row['cat']}] ({sentiment}) {row['review'][:60]}")
    print()


combined_search("续航好的手机")

In [ ]:
combined_search("适合孩子看的书")

## 6. 架构总结与扩展

### 三个工具的角色

| 工具 | 角色 | 本案例中的使用 |
|------|------|----------------|
| Daft | 数据处理引擎 | 读取 CSV/Lance、清洗、转换、`embed_text` 生成嵌入 |
| LanceDB | 数据湖 + 向量搜索 | 存储结构化数据、存储向量、语义搜索、混合过滤 |
| Ray (Demo 2) | 分布式加速 | `daft.context.set_runner_ray()` 即可加速大规模嵌入生成 |

### 生产环境扩展路径

```
+-------------------+     +-------------------+     +-------------------+
| Daft + Ray Runner |     | Lance on MinIO/S3 |     | LanceDB Search    |
| (K8s Workers)     | --> | (Shared Storage)  | <-- | (Single Node)     |
+-------------------+     +-------------------+     +-------------------+
```

- **计算扩展**：Daft 切换 Ray Runner，Ray on K8s 弹性伸缩（详见 Demo 2）
- **存储扩展**：Lance 文件放 S3 或 MinIO（本地部署的 S3 兼容存储），多节点共享数据
- **搜索扩展**：LanceDB 开源版单机即可满足大多数场景；更大规模可考虑 LanceDB Cloud

本地没有 S3？用 MinIO 一个容器即可模拟：
```bash
docker run -p 9000:9000 minio/minio server /data
```
代码只需将路径从 `"../lancedb_data"` 改为 `"s3://bucket/lancedb_data"`。

## 7. 清理

In [ ]:
for name in ["products_raw", "reviews_raw", "products", "reviews"]:
    db.drop_table(name, ignore_missing=True)
print(f"剩余表: {db.table_names()}")